In [ ]:
import apache_log_parser
from rich.progress import track
import pandas as pd
import user_db
import datetime

In [ ]:
log_file = open('data/access.log', 'r')
log = []

log = [line for line in log_file]

log_file.close()

In [ ]:
line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")
parsed_log = [line_parser(line) for line in track(log)]

print(f'Record Count: {len(parsed_log)}')

In [ ]:
# Keep commented out unless you want to reset the whole db
#user_db.reset_db()

In [ ]:
valid_pages = {'', 'about', 'search', 'explore', 'rankings', 'trail-rankings', 'lift-rankings', 'map', 'interactive-map'}

count = 0
previous_timestamp = datetime.datetime(1970, 1, 1)
tuple_list = []

for line in track(parsed_log):
    count += 1
    
    if 'request_url_path' not in line:
        continue

    page_visited = line['request_url_path']
    
    if '.' in page_visited or '/' not in page_visited:
        continue
    
    page_category = page_visited.split('/')[1]
    
    if page_category not in valid_pages:
        continue

    page_category = '/' + page_category
    timestamp = line['time_received_datetimeobj']
    ip = line['remote_host']

    parameters = line['request_url_query_dict']
    if parameters:
        parameters = str({key: value[0] for key, value in parameters.items()})
    else:
        parameters = None

    # Use microsecond-free versions once
    timestamp_no_micro = timestamp.replace(microsecond=0)
    previous_timestamp_no_micro = previous_timestamp.replace(microsecond=0)

    if timestamp_no_micro < previous_timestamp_no_micro:
        timestamp = previous_timestamp + datetime.timedelta(milliseconds=100)
    elif timestamp_no_micro == previous_timestamp_no_micro:
        timestamp = previous_timestamp + datetime.timedelta(milliseconds=1)

    tuple_list.append((timestamp, ip, page_category, page_visited, parameters))
    #user_db.add_log(ip, page_category, page_visited, parameters, timestamp)
    previous_timestamp = timestamp

    # every 500000 records, flush to db
    if count > 500000:
        user_db.bulk_add_log(tuple_list)
        count = 0
        tuple_list = []

user_db.bulk_add_log(tuple_list)